In [1]:
import cv2
import numpy as np
import imutils

img = cv2.imread('3.jpg')
img = imutils.resize(img,width=1000)
cv2.imshow("img",img)
cv2.waitKey()

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow("gray",gray)
cv2.waitKey()
cv2.destroyAllWindows()

res=gray

In [2]:
thresh = cv2.adaptiveThreshold(res,255,0,1,19,2)
cv2.imshow("thresh",thresh)
cv2.waitKey()
contour,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

max_area = 0
best_cnt = None
for cnt in contour:
    area = cv2.contourArea(cnt)
    if area > 1000:
        if area > max_area:
            max_area = area
            best_cnt = cnt
            
mask = np.zeros((gray.shape),np.uint8)
cv2.drawContours(mask,[best_cnt],0,255,-1)
cv2.imshow("mask",mask)
cv2.waitKey()
cv2.drawContours(mask,[best_cnt],0,0,2)
cv2.imshow("mask2",mask)
cv2.waitKey()

res2 = cv2.bitwise_and(res,mask)
cv2.imshow("res2",res2)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
cv2.imshow("res2",res2)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
kernelx = cv2.getStructuringElement(cv2.MORPH_RECT,(2,10))

dx = cv2.Sobel(res2,cv2.CV_16S,1,0)
dx = cv2.convertScaleAbs(dx)
cv2.imshow("dx",dx)
cv2.waitKey()

close = cv2.morphologyEx(dx,cv2.MORPH_DILATE,kernelx,iterations = 1)
cv2.imshow("close",close)
cv2.waitKey()

contour, hier = cv2.findContours(close,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
for cnt in contour:
    x,y,w,h = cv2.boundingRect(cnt)
    if h/w > 10:
        cv2.drawContours(close,[cnt],0,255,-1)
    else:
        cv2.drawContours(close,[cnt],0,0,-1)
        
close = cv2.morphologyEx(close,cv2.MORPH_CLOSE,None,iterations = 2)
closex = close.copy()
cv2.imshow("closex",closex)
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
kernely = cv2.getStructuringElement(cv2.MORPH_RECT,(10,2))

dy = cv2.Sobel(res2,cv2.CV_16S,0,1)
dy = cv2.convertScaleAbs(dy)
cv2.imshow("dy",dy)
cv2.waitKey()

close = cv2.morphologyEx(dy,cv2.MORPH_DILATE,kernely)
cv2.imshow("close",close)
cv2.waitKey()

contour, hier = cv2.findContours(close,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
for cnt in contour:
    x,y,w,h = cv2.boundingRect(cnt)
    if w/h > 10:
        cv2.drawContours(close,[cnt],0,255,-1)
    else:
        cv2.drawContours(close,[cnt],0,0,-1)

close = cv2.morphologyEx(close,cv2.MORPH_DILATE,None,iterations = 2)
closey = close.copy()
cv2.imshow("closey",closey)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
res = cv2.bitwise_and(closex,closey)
cv2.imshow("res",res)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
contour, hier = cv2.findContours(res,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
centroids = []
for cnt in contour:
    mom = cv2.moments(cnt)
    (x,y) = int(mom['m10']/mom['m00']), int(mom['m01']/mom['m00'])
    cv2.circle(img,(x,y),4,(0,255,0),-1)
    cv2.imshow("img",img)
    cv2.waitKey()
    centroids.append((x,y))
cv2.destroyAllWindows()

In [8]:
centroids = np.array(centroids,dtype = int)
c = centroids.reshape((100,2))
c2 = c[np.argsort(c[:,1])]

b = np.vstack([c2[i*10:(i+1)*10][np.argsort(c2[i*10:(i+1)*10,0])] for i in range(10)])
bm = b.reshape((10,10,2))

In [9]:
def find_k_nearest(model,img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,100,240,cv2.THRESH_BINARY_INV)
    cv2.imshow("thresh",thresh)

    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    best_cnt=None
    max_area = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            best_cnt = cnt

    [x,y,w,h] = cv2.boundingRect(best_cnt)

    if  h>28:
        roi = thresh[y:y+h,x:x+w]
        roismall = cv2.resize(roi,(30,30))
        cv2.imshow("roismall",roismall)
        roismall = roismall.reshape((1,900))
        roismall = np.float32(roismall)
        retval, results, neigh_resp, dists = model.findNearest(roismall, k = 1)
        string = str(int((results[0][0])))
        return int((results[0][0]))

In [10]:
model = cv2.ml.KNearest_load("lallo.model")

In [11]:
cv2.imshow("img",img)
cv2.waitKey()
sudoku = []

for i in range(90):
    if (i+1)%10 == 0:
        continue
    x = (i+1)//10
    y = (i+1)%10 - 1
    my_image = img[bm[x][y][1]:bm[x+1][y+1][1],bm[x][y][0]:bm[x+1][y+1][0]]
    w , h , _ = my_image.shape
    my_image = my_image[w//8:w-w//8,w//8:w-w//8]
    find_k_nearest(model,my_image)
    number = find_k_nearest(model,my_image)
    if number is None:
        sudoku.append(0)
    else:
        sudoku.append(number)
    cv2.imshow("my_image",my_image)
    cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
sudoku = np.array(sudoku)
sudoku = sudoku.reshape(9,9)
sudoku

array([[0, 0, 1, 3, 0, 2, 0, 0, 0],
       [0, 0, 3, 0, 0, 7, 0, 4, 5],
       [0, 0, 7, 0, 0, 0, 0, 0, 9],
       [0, 0, 6, 5, 0, 0, 0, 7, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 9, 0, 0, 0, 1, 4, 0, 0],
       [5, 0, 0, 0, 0, 0, 9, 0, 0],
       [6, 1, 0, 2, 0, 0, 8, 0, 0],
       [0, 0, 0, 9, 0, 8, 5, 0, 0]])

In [13]:
def check(sudoku,i,j,k):
    for x in range(9):
        if (sudoku[x][j] == k) or (sudoku[i][x] == k):
            return False
        
    x = (i//3)*3
    y = (j//3)*3
    for q in range(x,x+3):
        for w in range(y,y+3):
            if sudoku[q][w] == k:
                return False
    return True

In [14]:
def solve(sudoku):
    global n
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0:
                for k in range(1,10):
                    if check(sudoku,i,j,k):
                        sudoku[i][j]=k
                        solve(sudoku)
                        sudoku[i][j]=0
                return
    print("answer",n)
    print(np.matrix(sudoku),"\n")
    n=n+1

In [15]:
n=1
solve(sudoku)

answer 1
[[4 5 1 3 9 2 7 8 6]
 [9 2 3 8 6 7 1 4 5]
 [8 6 7 1 5 4 3 2 9]
 [1 3 6 5 4 9 2 7 8]
 [2 4 5 7 8 3 6 9 1]
 [7 9 8 6 2 1 4 5 3]
 [5 8 2 4 3 6 9 1 7]
 [6 1 9 2 7 5 8 3 4]
 [3 7 4 9 1 8 5 6 2]] 

